In [47]:
import datetime as dt
import ast
import json
from collections import Counter

import numpy as np
import pandas as pd
import string
import re

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Flatten, Dense, concatenate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import statsmodels.api as sm

In [48]:
df = pd.read_csv('new_cars.csv')

In [49]:
### Drop unncessary columns
columns_to_drop = ['link', 'stock_number', 'vin', 'deal_gauge', 'exterior_color', 'mpg', 'fuel_type', 'engine', 'price_history', 'vehicle_history', 'listing_title', 'interior_color', 'Open recall', 'Accidents or damage']
# add Open recall back once cleaned
# add Accidents or damage back once cleaned
df = df.drop(columns=columns_to_drop)

In [50]:
### Convert data types
boolean_columns = ['1-owner vehicle', 'Personal use only', 'Clean title']

for column in boolean_columns:
    df[column] = df[column].map({'Yes': True, 'No': False}).astype(bool)

In [51]:
# Check for NAs
missing_counts = df.isna().sum()
print(missing_counts[missing_counts > 0])

seller_name                  45
listed_date                1523
price_change               1523
price_change_percentage    1523
Cylinders                    74
Liters                      106
exterior_color_1            227
exterior_color_2           5992
interior_color_1            539
interior_color_2           5799
low mpg                     427
high mpg                    427
dtype: int64


In [52]:
# Drop rows with a low number of missing values
columns_with_missing_values = ['seller_name', 'Cylinders', 'Liters', 'exterior_color_1', 'low mpg', 'high mpg', 'interior_color_1']
df = df.dropna(subset=columns_with_missing_values)

# Drop exterior_color_2 since it only has 12 values
df = df.drop(columns=['exterior_color_2'])
# Drop interior_color_2 since it only has 203 values
df = df.drop(columns=['interior_color_2'])

In [53]:
# Convert listed_date to Days Avaliable
current_date = pd.Timestamp(dt.datetime.now().date())
df['listed_date'] = pd.to_datetime(df['listed_date'])
df['Days Avaliable'] = (current_date - df['listed_date']).dt.days
df = df.drop('listed_date', axis=1)
df.head()

,listing_mileage,primary_price,drivetrain,transmission,seller_name,1-owner vehicle,Personal use only,Clean title,year,make,...,price_change,price_change_percentage,Cylinders,Liters,Boosted,exterior_color_1,interior_color_1,low mpg,high mpg,Days Avaliable
0,79497,17899,Front-wheel Drive,6-Speed Automatic,ETHIO Motors,True,False,True,2018,Hyundai,...,-2400.0,-11.823243,4.0,2.4,False,blue,beige,21.0,27.0,785.0
1,80326,27900,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,True,True,2017,RAM,...,-3085.0,-9.956431,8.0,5.7,False,silver,black,16.0,23.0,610.0
2,92959,18950,Front-wheel Drive,Automatic CVT,Custom Cars West,False,True,True,2019,Nissan,...,-1030.0,-5.155155,4.0,1.6,False,black,gray,31.0,36.0,578.0
3,59176,39950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,2019,RAM,...,-573.0,-1.414012,8.0,5.7,False,gray,black,19.0,24.0,532.0
4,135385,26950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,True,True,2015,Jeep,...,-3000.0,-10.016694,6.0,3.6,False,blue,black,16.0,21.0,663.0


In [54]:
# Drop missing price history data. ~25% of data.
columns_with_missing_values = ['price_change', 'price_change_percentage']
df = df.dropna(subset=columns_with_missing_values)

In [55]:
df.head()

,listing_mileage,primary_price,drivetrain,transmission,seller_name,1-owner vehicle,Personal use only,Clean title,year,make,...,price_change,price_change_percentage,Cylinders,Liters,Boosted,exterior_color_1,interior_color_1,low mpg,high mpg,Days Avaliable
0,79497,17899,Front-wheel Drive,6-Speed Automatic,ETHIO Motors,True,False,True,2018,Hyundai,...,-2400.0,-11.823243,4.0,2.4,False,blue,beige,21.0,27.0,785.0
1,80326,27900,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,True,True,2017,RAM,...,-3085.0,-9.956431,8.0,5.7,False,silver,black,16.0,23.0,610.0
2,92959,18950,Front-wheel Drive,Automatic CVT,Custom Cars West,False,True,True,2019,Nissan,...,-1030.0,-5.155155,4.0,1.6,False,black,gray,31.0,36.0,578.0
3,59176,39950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,2019,RAM,...,-573.0,-1.414012,8.0,5.7,False,gray,black,19.0,24.0,532.0
4,135385,26950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,True,True,2015,Jeep,...,-3000.0,-10.016694,6.0,3.6,False,blue,black,16.0,21.0,663.0


In [56]:
df.shape

(3578, 21)

### outcome variable: primary_price
### ocluded variables: price_change	price_change_percentage

In [57]:
# Drop 'price_change' and 'price_change_percentage' to avoid data leakage
columns_to_drop = ['price_change', 'price_change_percentage']
df = df.drop(columns=columns_to_drop)

## Categorical variables

In [58]:
categorical_columns = ['seller_name', 'drivetrain', 'transmission', 'make', 'model', 'exterior_color_1', 'interior_color_1']

# Check how many unique values there are in categorical_columns
for column in categorical_columns:
    unique_values = df[column].nunique()
    print(f"Unique values for {column}:", unique_values)

# Models has a large number of unique values
# seller_name, transmission, make, model have a large number of unique values
# Consider embedding these instead of one-hot encode

Unique values for seller_name: 121
Unique values for drivetrain: 5
Unique values for transmission: 81
Unique values for make: 47
Unique values for model: 1327
Unique values for exterior_color_1: 13
Unique values for interior_color_1: 11


In [59]:
X = df.drop('primary_price', axis=1)  # Features
y = df['primary_price']  # Target

In [60]:
X.dtypes

listing_mileage        int64
drivetrain            object
transmission          object
seller_name           object
1-owner vehicle         bool
Personal use only       bool
Clean title             bool
year                   int64
make                  object
model                 object
Cylinders            float64
Liters               float64
Boosted                 bool
exterior_color_1      object
interior_color_1      object
low mpg              float64
high mpg             float64
Days Avaliable       float64
dtype: object

In [61]:
y.dtype

dtype('int64')

In [62]:
### One-hot Encode 
X = pd.get_dummies(X, columns=categorical_columns)

In [63]:
X.shape

(3578, 1616)

In [64]:
# Check that X doesn't have object data
unsupported_types = []

for column in X.columns:
    dtype = X[column].dtype
    if dtype == 'object':
        unsupported_types.append((column, dtype))

print("Unsupported Data Types:")
for column, dtype in unsupported_types:
    print(f"{column}: {dtype}")


Unsupported Data Types:


In [65]:
df.head()

,listing_mileage,primary_price,drivetrain,transmission,seller_name,1-owner vehicle,Personal use only,Clean title,year,make,model,Cylinders,Liters,Boosted,exterior_color_1,interior_color_1,low mpg,high mpg,Days Avaliable
0,79497,17899,Front-wheel Drive,6-Speed Automatic,ETHIO Motors,True,False,True,2018,Hyundai,Santa Fe Sport 2.4L,4.0,2.4,False,blue,beige,21.0,27.0,785.0
1,80326,27900,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,True,True,2017,RAM,1500 Express,8.0,5.7,False,silver,black,16.0,23.0,610.0
2,92959,18950,Front-wheel Drive,Automatic CVT,Custom Cars West,False,True,True,2019,Nissan,Kicks SR,4.0,1.6,False,black,gray,31.0,36.0,578.0
3,59176,39950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,2019,RAM,1500 Laramie,8.0,5.7,False,gray,black,19.0,24.0,532.0
4,135385,26950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,True,True,2015,Jeep,Wrangler Unlimited Rubicon,6.0,3.6,False,blue,black,16.0,21.0,663.0


## Split data into 80% training set and 20% validation set.

In [66]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert all columns to float
X_train = X_train.astype(float)
X_val = X_val.astype(float)
y_train = y_train.astype(float)
y_val = y_val.astype(float)

# Convert data to numpy array
X_train = X_train.to_numpy()
X_val = X_val.to_numpy()
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

In [67]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)

X_train shape: (2862, 1616)
X_val shape: (716, 1616)
y_train shape: (2862,)
y_val shape: (716,)


In [68]:
# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.Tensor(X_train)
y_train_tensor = torch.Tensor(y_train)
X_val_tensor = torch.Tensor(X_val)
y_val_tensor = torch.Tensor(y_val)

## Deep Neural Net

In [69]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(128, 64)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.5)
        self.fc4 = nn.Linear(64, 1)  # Output layer for regression

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.dropout3(x)
        x = self.fc4(x)
        return x

# Create the model
input_size = X_train_tensor.shape[1]
model = NeuralNetwork(input_size)

# Loss function for regression
criterion = nn.MSELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 100
for epoch in range(epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.view(-1, 1))
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Calculate training accuracy
    with torch.no_grad():
        train_outputs = model(X_train_tensor)
        train_loss = criterion(train_outputs, y_train_tensor.view(-1, 1))
    
    # Print training loss
    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {train_loss.item()}')


Epoch [50/100], Training Loss: 1145542400.0
Epoch [100/100], Training Loss: 467593632.0


In [70]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_model(model, criterion, X, y):
    # Forward pass
    outputs = model(X)
    
    # Calculate loss
    loss = criterion(outputs, y.view(-1, 1))
    
    # Convert predictions and targets to numpy arrays
    y_pred = outputs.detach().numpy()
    y_true = y.numpy()
    
    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    
    return loss.item(), mae, mse, rmse, r2

# After the training loop

# Evaluate the model on the training set
train_loss, train_mae, train_mse, train_rmse, train_r2 = evaluate_model(model, criterion, X_train_tensor, y_train_tensor)

# Evaluate the model on the validation set
val_loss, val_mae, val_mse, val_rmse, val_r2 = evaluate_model(model, criterion, X_val_tensor, y_val_tensor)

# Print the evaluation metrics
print(f"Training:\nLoss: {train_loss:.4f}\nMAE: {train_mae:.4f}\nRMSE: {train_rmse:.4f}\nR^2: {train_r2:.4f}\n")
print(f"Validation:\nLoss: {val_loss:.4f}\nMAE: {val_mae:.4f}\nRMSE: {val_rmse:.4f}\nR^2: {val_r2:.4f}")

Training:
Loss: 475860896.0000
MAE: 12260.6816
RMSE: 21814.2363
R^2: 0.0984

Validation:
Loss: 346814208.0000
MAE: 11973.7148
RMSE: 18622.9492
R^2: 0.0535


## Linear Regression

In [71]:
# Setup model
m = LinearRegression()
# Give input and output data
m.fit(X_train, y_train)
print('Training accuracy (the coefficient of determination): {:.3f}'.format(m.score(X_train, y_train)))

Training accuracy (the coefficient of determination): 0.987


## Test the model on the test data

In [72]:
prediction = m.predict(X_val)

In [73]:
print('Testing accuracy by using score function: {:.3f}'.format(m.score(X_val, y_val)))
print('Testing accuracy by using r2_score meric: {:.3f}'.format(r2_score(y_val, m.predict(X_val))))

Testing accuracy by using score function: 0.719
Testing accuracy by using r2_score meric: 0.719


In [74]:
mean_absolute_error(y_val, prediction)

4980.197615977653

## Add statsmodel for p-value

In [75]:
X_train_with_constant = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_with_constant).fit()
# Get the summary of the regression

#### Change the variables to the actual variable name

In [76]:
xnames = ['const']
xnames.extend(list(X.columns))

In [77]:
exog_names = model.model.exog_names

In [78]:
dic = dict(zip(exog_names, xnames))

In [79]:
def rename_vars(vname): 
    to_ret = vname
    for orig_vname in list(dic.keys()):
        if vname == orig_vname:
            to_ret = dic[orig_vname] 
    return to_ret

def rename_ols(reg): 
    for i in range(len(reg)): 
        reg[i] = rename_vars(reg[i])

In [80]:
rename_ols(model.model.exog_names)

In [81]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.975
Method:                 Least Squares   F-statistic:                     82.69
Date:                Wed, 15 Nov 2023   Prob (F-statistic):               0.00
Time:                        20:23:12   Log-Likelihood:                -26556.
No. Observations:                2862   AIC:                         5.588e+04
Df Residuals:                    1476   BIC:                         6.414e+04
Df Model:                        1385                                         
Covariance Type:            nonrobust                                         
                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------